## Setup for google colab

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [2]:
%cd /content/drive/My Drive/SA/Code/spoof_detection_deep_features/WaveletCNN

/content/drive/My Drive/SA/Code/spoof_detection_deep_features/WaveletCNN


## Run code from here

In [18]:
!pip install keras==2.1.5

  Using cached Keras-2.1.5-py2.py3-none-any.whl (334 kB)
  Attempting uninstall: keras
    Found existing installation: Keras 2.3.1
    Uninstalling Keras-2.3.1:
      Successfully uninstalled Keras-2.3.1


In [3]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
 
# The GPU id to use, usually either "0" or "1";
os.environ["CUDA_VISIBLE_DEVICES"]="1";  

In [ ]:
import tensorflow as tf
sess = tf.Session()

from keras import backend as K
K.set_session(sess)

In [1]:
import warnings
warnings.simplefilter(action='ignore', category=Warning)
import tensorflow as tf
import pandas as pd
import math
import numpy as np
from tensorflow.python.keras.layers import MaxPooling1D,MaxPooling2D,AveragePooling1D, Conv1D, LeakyReLU, BatchNormalization, Dense, Flatten, concatenate, Activation
from tensorflow.python.keras.layers import InputLayer, Input, Layer, Lambda, Dropout
from keras.utils import to_categorical
from tensorflow.python.keras.models import Model
from sklearn.preprocessing import StandardScaler
import tfwavelets
import keras
import tensorflow.contrib.slim as slim


Using TensorFlow backend.


In [4]:
def generator(data, labels, batch_size):
    """
    Yields the next training batch.
    Suppose `samples` is an array [[audio1,label1], [audio2,label2],...].
    """
    total_batches = int(data.shape[0])
    for i in range(total_batches):
        X_train,y_train = create_batches_rnd(data, labels, batch_size)
        yield X_train, y_train

In [5]:
def generator_val(data, labels, batch_size):
    """
    Yields the next training batch.
    Suppose `samples` is an array [[audio1,label1], [audio2,label2],...].
    """
    total_batches = int(data.shape[0])
    for i in range(total_batches):
        X_train,y_train = create_batches_rnd(data, labels, batch_size)
        yield X_train, y_train

In [6]:
def create_batches_rnd(data,labels,batch_size):
    wlen = 3200
    fact_amp = 0.2
    # Initialization of the minibatch (batch_size,[0=>x_t,1=>x_t+N,1=>random_samp])
    sig_batch=np.zeros([batch_size,wlen])
    lab_batch=[]
    signal_id_arr=np.random.randint(data.shape[0], size=batch_size)
    rand_amp_arr = np.random.uniform(1.0-fact_amp,1+fact_amp,batch_size)
    for i in range(batch_size): 
        signal = data[signal_id_arr[i]]
        # accesing to a random chunk
        signal_len=signal.shape[0]
        signal_beg=np.random.randint(signal_len-wlen-1) #randint(0, snt_len-2*wlen-1)
        signal_end=signal_beg+wlen
        sig_batch[i,:]=signal[signal_beg:signal_end]*rand_amp_arr[i]
        y=labels[signal_id_arr[i]]
        lab_batch.append(y)
    a, b = np.shape(sig_batch)
    sig_batch = sig_batch.reshape((a, b, 1))
    return sig_batch, np.array(lab_batch)

In [7]:
def res_conv_block(X,in_channels,out_channels,stage,block,dilation=1):

    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    X_shortcut = X
    
    X = BatchNormalization(name=bn_name_base+'a')(X)
    X = Activation('relu')(X)
    X = Conv1D(in_channels, 3, padding='valid',use_bias = False, name= conv_name_base+'a')(X)
    X = BatchNormalization(name=bn_name_base+'b')(X)
    X = Activation('relu')(X)
    X = Conv1D(in_channels, 3, padding='valid',use_bias = False, name= conv_name_base+'b')(X)
    print(X.shape)
    paddings = tf.constant([[0, 0],   # the batch size dimension
                          [2, 2],   # top and bottom of image
                          [0, 0]])  # the channels dimension
    X = Lambda(lambda x: tf.pad(x, paddings, mode='CONSTANT',
                        constant_values=0.0))(X)
    X = concatenate([X , X_shortcut])
    X = BatchNormalization(name = bn_name_base+'c')(X)
    X = Activation('relu')(X)
    X = Conv1D(out_channels, 3, padding='valid',use_bias = False, dilation_rate = dilation, name = conv_name_base+'c')(X)

    return X



In [8]:
def dwt(x,db2):
    # x1_ = tf.placeholder(tf.float32, shape=(None,3200,3), name= 'x1')
    dwt = tfwavelets.nodes.dwt1d(x,db2,1)
    return dwt

    


def iwt(x,db2):
    
    idwt = tfwavelets.nodes.idwt1d(x,db2,1)
    return idwt
db2 = tfwavelets.dwtcoeffs.Wavelet(
    tfwavelets.dwtcoeffs.Filter(np.array([-0.12940952255092145,
                     0.22414386804185735,
                     0.836516303737469,
                     0.48296291314469025]), 3),
    tfwavelets.dwtcoeffs.Filter(np.array([-0.48296291314469025,
                     0.836516303737469,
                     -0.22414386804185735,
                     -0.12940952255092145]), 0),
    tfwavelets.dwtcoeffs.Filter(np.array([0.48296291314469025,
                     0.836516303737469,
                     0.22414386804185735,
                     -0.12940952255092145]), 0),
    tfwavelets.dwtcoeffs.Filter(np.array([-0.12940952255092145,
                     -0.22414386804185735,
                     0.836516303737469,
                     -0.48296291314469025]), 3)
)

class DWT_Pooling(Layer):
    """
    # Input shape :
        
            4D tensor of shape: (batch_size, signal, channels)
        
            
    # Output shape
        
            4D tensor of shape: (batch_size, signal/2, channels*4)
        
    """

    def __init__(self,**kwargs):
        super(DWT_Pooling, self).__init__(**kwargs)

    def build(self, input_shape):
        super(DWT_Pooling, self).build(input_shape) 

    def call(self, x):
        return dwt(x,db2)

    def compute_output_shape(self, input_shape):
        
        return (input_shape[0], input_shape[1]//2, input_shape[2]*4)


class IWT_UpSampling(Layer):
    """
    # Input shape :
        
            4D tensor of shape: (batch_size, signal, channels)
        
    # Output shape
        
            4D tensor of shape: (batch_size, singal*2, channels/4)
        
    """

    def __init__(self, **kwargs):
        super(IWT_UpSampling, self).__init__(**kwargs)

    def build(self, input_shape):
        super(IWT_UpSampling, self).build(input_shape) 

    def call(self, x):
        return iwt(x,db2)

    def compute_output_shape(self, input_shape):
        
        return (input_shape[0], input_shape[1]*2, input_shape[2]//4)

In [10]:
# def unetWavelet(input_size = (3200,1)):


input_shape = None,3200,1


inputs = tf.placeholder(tf.float32, shape=input_shape, name= 'the_input')
input_l1 = DWT_Pooling()(inputs)
input_l2 = DWT_Pooling()(input_l1)
input_l3 = DWT_Pooling()(input_l2)
input_l4 = DWT_Pooling()(input_l3)
# level one decomposition starts
conv_1 = Conv1D(filters=64, kernel_size= 3, padding="same")(input_l1)
norm_1 = BatchNormalization(name='norm_1')(conv_1)
relu_1 = Activation('relu', name='relu_1')(norm_1)

conv_1_2 = Conv1D(filters=64, kernel_size= 3, padding="same")(relu_1)#strides = 2, padding="same")(relu_1)
norm_1_2 = BatchNormalization(name='norm_1_2')(conv_1_2)
relu_1_2 = Activation('relu', name='relu_1_2')(norm_1_2)

# level two decomposition starts
conv_a = Conv1D(filters=64, kernel_size= 3, padding="same")(input_l2)
norm_a = BatchNormalization(name='norm_a')(conv_a)
relu_a = Activation('relu', name='relu_a')(norm_a)

# concate level one and level two decomposition
concate_level_2 = concatenate([relu_1_2, relu_a])
conv_2 = Conv1D(filters=128, kernel_size= 3, padding="same")(concate_level_2)
norm_2 = BatchNormalization(name='norm_2')(conv_2)
relu_2 = Activation('relu', name='relu_2')(norm_2)

conv_2_2 = Conv1D(filters=128, kernel_size= 3, padding="same")(relu_2)#strides = 2, padding="same")(relu_2)
norm_2_2 = BatchNormalization(name='norm_2_2')(conv_2_2)
relu_2_2 = Activation('relu', name='relu_2_2')(norm_2_2)

# level three decomposition starts 
conv_b = Conv1D(filters=64, kernel_size= 3, padding="same")(input_l3)
norm_b = BatchNormalization(name='norm_b')(conv_b)
relu_b = Activation('relu', name='relu_b')(norm_b)

conv_b_2 = Conv1D(filters=128, kernel_size= 3, padding="same")(relu_b)
norm_b_2 = BatchNormalization(name='norm_b_2')(conv_b_2)
relu_b_2 = Activation('relu', name='relu_b_2')(norm_b_2)

# concate level two and level three decomposition 
concate_level_3 = concatenate([relu_2_2, relu_b_2])
conv_3 = Conv1D(filters=256, kernel_size= 3, padding="same")(concate_level_3)
norm_3 = BatchNormalization(name='nomr_3')(conv_3)
relu_3 = Activation('relu', name='relu_3')(norm_3)

conv_3_2 = Conv1D(filters=256, kernel_size= 3, padding="same")(relu_3)#strides = 2, padding="same")(relu_3)
norm_3_2 = BatchNormalization(name='norm_3_2')(conv_3_2)
relu_3_2 = Activation('relu', name='relu_3_2')(norm_3_2)

# level four decomposition start
conv_c = Conv1D(filters=64, kernel_size= 3, padding="same")(input_l4)
norm_c = BatchNormalization(name='norm_c')(conv_c)
relu_c = Activation('relu', name='relu_c')(norm_c)

conv_c_2 = Conv1D(filters=256, kernel_size= 3, padding="same")(relu_c)
norm_c_2 = BatchNormalization(name='norm_c_2')(conv_c_2)
relu_c_2 = Activation('relu', name='relu_c_2')(norm_c_2)

conv_c_3 = Conv1D(filters=256, kernel_size= 3, padding="same")(relu_c_2)
norm_c_3 = BatchNormalization(name='norm_c_3')(conv_c_3)
relu_c_3 = Activation('relu', name='relu_c_3')(norm_c_3)

# concate level level three and level four decomposition
concate_level_4 = concatenate([relu_3_2, relu_c_3])
conv_4 = Conv1D(filters=256, kernel_size= 3, padding="same")(concate_level_4)
norm_4 = BatchNormalization(name='norm_4')(conv_4)
relu_4 = Activation('relu', name='relu_4')(norm_4)

conv_4_2 = Conv1D(filters=256, kernel_size= 3, strides = 2, padding="same")(relu_4)
norm_4_2 = BatchNormalization(name='norm_4_2')(conv_4_2)
relu_4_2 = Activation('relu', name='relu_4_2')(norm_4_2)

conv_5_1 = Conv1D(filters=128, kernel_size= 3, padding="same")(relu_4_2)
norm_5_1 = BatchNormalization(name='norm_5_1')(conv_5_1)
relu_5_1 = Activation('relu', name='relu_5_1')(norm_5_1)

res_conv_1 = res_conv_block(relu_5_1, 128, 16, 1, 'a', 4)
res_conv_2 = res_conv_block(res_conv_1, 16, 8, 2, 'a', 8)
res_conv_3 = res_conv_block(res_conv_2, 8, 4, 3, 'a', 16)
res_conv_4 = res_conv_block(res_conv_3, 4, 2, 4, 'a', 32)
res_conv_5 = res_conv_block(res_conv_4, 2, 1, 5, 'a', 64)

res_norm = BatchNormalization(1,name='res_norm')(res_conv_5)
res_relu = Activation('relu')(res_norm)


pool_5_1 = AveragePooling1D(pool_size=7 , strides=1, padding='same', name='avg_pool_5_1')(res_relu)
flat_5_1 = Flatten(name='flat_5_1')(pool_5_1) 

fc_5 = Dense(2048, name='fc_5')(flat_5_1)
norm_5 = BatchNormalization(name='norm_5')(fc_5)
relu_5 = Activation('relu', name='relu_5')(norm_5)
drop_5 = Dropout(0.5, name='drop_5')(relu_5)

fc_6 = Dense(2048, name='fc_6')(drop_5)
norm_6 = BatchNormalization(name='norm_6')(fc_6)
relu_6 = Activation('relu', name='relu_6')(norm_6)
drop_6 = Dropout(0.5, name='drop_6')(relu_6)

output = Dense(2, activation='softmax', name='fc_7')(drop_6)
# model = Model(input = inputs, output = output)
    
# return model

(?, 1596, 128)
(?, 1588, 16)
(?, 1572, 8)
(?, 1540, 4)
(?, 1476, 2)


In [11]:
def model_summary():
    model_vars = tf.trainable_variables()
    slim.model_analyzer.analyze_vars(model_vars, print_info=True)

model_summary()

---------
Variables: name (type shape) [size]
---------
conv1d/kernel:0 (float32_ref 3x1x64) [192, bytes: 768]
conv1d/bias:0 (float32_ref 64) [64, bytes: 256]
norm_1/gamma:0 (float32_ref 64) [64, bytes: 256]
norm_1/beta:0 (float32_ref 64) [64, bytes: 256]
conv1d_1/kernel:0 (float32_ref 3x64x64) [12288, bytes: 49152]
conv1d_1/bias:0 (float32_ref 64) [64, bytes: 256]
norm_1_2/gamma:0 (float32_ref 64) [64, bytes: 256]
norm_1_2/beta:0 (float32_ref 64) [64, bytes: 256]
conv1d_2/kernel:0 (float32_ref 3x1x64) [192, bytes: 768]
conv1d_2/bias:0 (float32_ref 64) [64, bytes: 256]
norm_a/gamma:0 (float32_ref 64) [64, bytes: 256]
norm_a/beta:0 (float32_ref 64) [64, bytes: 256]
conv1d_3/kernel:0 (float32_ref 3x128x128) [49152, bytes: 196608]
conv1d_3/bias:0 (float32_ref 128) [128, bytes: 512]
norm_2/gamma:0 (float32_ref 128) [128, bytes: 512]
norm_2/beta:0 (float32_ref 128) [128, bytes: 512]
conv1d_4/kernel:0 (float32_ref 3x128x128) [49152, bytes: 196608]
conv1d_4/bias:0 (float32_ref 128) [128, byte

In [12]:
labels = tf.placeholder(tf.float32, shape=(None,2))

from keras.metrics import categorical_accuracy as accuracy


with tf.name_scope('Accuracy'):
    acc= tf.reduce_mean(accuracy(labels, output))

from keras.objectives import categorical_crossentropy
with tf.name_scope('Loss'):
    loss = tf.reduce_mean(categorical_crossentropy(labels, output))

In [13]:
X_train = np.load("/home/rohita/rohit/spoof/npy_data_asvspoof/ASVspoof2019_train_train.npy")
y_train = np.load("/home/rohita/rohit/spoof/npy_data_asvspoof/ASVspoof2019_train_train_labels.npy")


y_train1 = list()
for i in y_train:
    if i == b'bonafide':
        y_train1.append(1)
    else:
        y_train1.append(0)
y_train = to_categorical(y_train)
# X_dev_train = np.load("/home/rohita/rohit/spoof/npy_data_asvspoof/ASVspoof2019_dev_train.npy")
# X_dev_val = np.load("/home/rohita/rohit/spoof/npy_data_asvspoof/ASVspoof2019_dev_val.npy")
# y_dev_train = np.load("/home/rohita/rohit/spoof/npy_data_asvspoof/ASVspoof2019_dev_train_labels.npy")
# y_dev_val = np.load("/home/rohita/rohit/spoof/npy_data_asvspoof/ASVspoof2019_dev_val_labels.npy")
# dev_wpt_levels_data_train = np.load("/home/rohita/rohit/spoof/npy_data_asvspoof/ASVspoof2019_dev_wpt_levels_data_train.npy")
# dev_wpt_levels_data_val = np.load("/home/rohita/rohit/spoof/npy_data_asvspoof/ASVspoof2019_dev_wpt_levels_data_val.npy")
# wpt_levels_data_train = np.load("/home/rohita/rohit/spoof/npy_data_asvspoof/ASVspoof2019_train_wpt_levels_data_train.npy")
X_val = np.load("/home/rohita/rohit/spoof/npy_data_asvspoof/ASVspoof2019_dev_train.npy")
y_val = np.load("/home/rohita/rohit/spoof/npy_data_asvspoof/ASVspoof2019_dev_train_labels.npy")
# y_val = list(y_val)
# y_val1 = list()
# for i in y_val:
#     if i == b'bonafide':
#         y_val1.append(1)
#     else:
#         y_val1.append(0)
y_val = to_categorical(y_val)

In [ ]:
with tf.name_scope('RMSProp'):
    # Gradient Descent
    optimizer = tf.train.RMSPropOptimizer(0.001)
    train_step = optimizer.minimize(loss)
    # Op to calculate every variable gradient

tf.summary.scalar("loss", loss)
# Create a summary to monitor accuracy tensor
tf.summary.scalar("accuracy", acc)
# Create summaries to visualize weights
for var in tf.trainable_variables():
    tf.summary.histogram(var.name, var)
# Merge all summaries into a single op
merged_summary_op = tf.summary.merge_all()

# Initialize all variables
init_op = tf.global_variables_initializer()

batch_size = 32
logs_path = '/home/rohita/rohit/spoof/npy_data_asvspoof/WCNN'

# Run training loop
with sess.as_default():
    
    sess.run(init_op)
    
    train_summary_writer = tf.summary.FileWriter(logs_path+'/Train_whole_dataset_RMS',
                                            graph=tf.get_default_graph())
    val_summary_writer = tf.summary.FileWriter(logs_path+'/Val_whole_dataset_RMS')
    gen = generator(X_train, y_train, batch_size)
    gen_val = generator_val(X_val, y_val, batch_size)
    total_batch_train = int(X_train.shape[0]/batch_size)
    total_batch_val = int(X_val.shape[0]/batch_size)
    for epoch in range(100):
        for i in range(total_batch_train):
            X_batch, y_batch = next(gen)
            feed_dict = {inputs: X_batch, labels: y_batch, tf.keras.backend.learning_phase(): 1}
            sess.run(train_step,feed_dict)
            loss_train, acc_train, summary = (sess.run([loss, acc, merged_summary_op],feed_dict))
            train_summary_writer.add_summary(summary, epoch * total_batch_train + i)
            print("Epoch: "+str(epoch)+"step: "+str(i)+"Training loss: ",loss_train," ","Training accuracy"," ",acc_train)
        
#             loss_val,acc_val = (sess.run([loss,acc_value],feed_dict={inputs: X_batch, labels: y_batch, tf.keras.backend.learning_phase(): 0}))
#             print("Epoch: "+str(i)+"step: "+str(step)+"loss: ",loss_val," ","accuracy"," ",acc_val)
            
                    
        for i in range(total_batch_val):
            X_batch, y_batch = next(gen_val)
            loss_val, acc_val, summary = (sess.run([loss, acc, merged_summary_op],feed_dict={inputs: X_batch, labels: y_batch, tf.keras.backend.learning_phase(): 0}))
            val_summary_writer.add_summary(summary, epoch * total_batch_val + i)
            print("val loss: ",loss_val," ","val accuracy"," ",acc_val)
            

INFO:tensorflow:Summary name conv1d/kernel:0 is illegal; using conv1d/kernel_0 instead.
INFO:tensorflow:Summary name conv1d/bias:0 is illegal; using conv1d/bias_0 instead.
INFO:tensorflow:Summary name norm_1/gamma:0 is illegal; using norm_1/gamma_0 instead.
INFO:tensorflow:Summary name norm_1/beta:0 is illegal; using norm_1/beta_0 instead.
INFO:tensorflow:Summary name conv1d_1/kernel:0 is illegal; using conv1d_1/kernel_0 instead.
INFO:tensorflow:Summary name conv1d_1/bias:0 is illegal; using conv1d_1/bias_0 instead.
INFO:tensorflow:Summary name norm_1_2/gamma:0 is illegal; using norm_1_2/gamma_0 instead.
INFO:tensorflow:Summary name norm_1_2/beta:0 is illegal; using norm_1_2/beta_0 instead.
INFO:tensorflow:Summary name conv1d_2/kernel:0 is illegal; using conv1d_2/kernel_0 instead.
INFO:tensorflow:Summary name conv1d_2/bias:0 is illegal; using conv1d_2/bias_0 instead.
INFO:tensorflow:Summary name norm_a/gamma:0 is illegal; using norm_a/gamma_0 instead.
INFO:tensorflow:Summary name norm_

INFO:tensorflow:Summary name bn4a_branchb/gamma:0 is illegal; using bn4a_branchb/gamma_0 instead.
INFO:tensorflow:Summary name bn4a_branchb/beta:0 is illegal; using bn4a_branchb/beta_0 instead.
INFO:tensorflow:Summary name res4a_branchb/kernel:0 is illegal; using res4a_branchb/kernel_0 instead.
INFO:tensorflow:Summary name bn4a_branchc/gamma:0 is illegal; using bn4a_branchc/gamma_0 instead.
INFO:tensorflow:Summary name bn4a_branchc/beta:0 is illegal; using bn4a_branchc/beta_0 instead.
INFO:tensorflow:Summary name res4a_branchc/kernel:0 is illegal; using res4a_branchc/kernel_0 instead.
INFO:tensorflow:Summary name bn5a_brancha/gamma:0 is illegal; using bn5a_brancha/gamma_0 instead.
INFO:tensorflow:Summary name bn5a_brancha/beta:0 is illegal; using bn5a_brancha/beta_0 instead.
INFO:tensorflow:Summary name res5a_brancha/kernel:0 is illegal; using res5a_brancha/kernel_0 instead.
INFO:tensorflow:Summary name bn5a_branchb/gamma:0 is illegal; using bn5a_branchb/gamma_0 instead.
INFO:tensorflo

Epoch: 0step: 81Training loss:  0.59596205   Training accuracy   0.6875
Epoch: 0step: 82Training loss:  0.49367145   Training accuracy   0.8125
Epoch: 0step: 83Training loss:  0.93575   Training accuracy   0.5
Epoch: 0step: 84Training loss:  0.47595084   Training accuracy   0.90625
Epoch: 0step: 85Training loss:  0.83251286   Training accuracy   0.71875
Epoch: 0step: 86Training loss:  1.0412086   Training accuracy   0.6875
Epoch: 0step: 87Training loss:  0.61393785   Training accuracy   0.65625
Epoch: 0step: 88Training loss:  0.5201082   Training accuracy   0.65625
Epoch: 0step: 89Training loss:  0.68894726   Training accuracy   0.78125
Epoch: 0step: 90Training loss:  0.5174427   Training accuracy   0.84375
Epoch: 0step: 91Training loss:  0.96496886   Training accuracy   0.59375
Epoch: 0step: 92Training loss:  0.87335247   Training accuracy   0.59375
Epoch: 0step: 93Training loss:  0.5387987   Training accuracy   0.75
Epoch: 0step: 94Training loss:  0.7393055   Training accuracy   0.75

Epoch: 0step: 195Training loss:  0.54567426   Training accuracy   0.71875
Epoch: 0step: 196Training loss:  0.5386334   Training accuracy   0.84375
Epoch: 0step: 197Training loss:  0.3184902   Training accuracy   0.875
Epoch: 0step: 198Training loss:  0.46780446   Training accuracy   0.78125
Epoch: 0step: 199Training loss:  0.50405586   Training accuracy   0.78125
Epoch: 0step: 200Training loss:  0.47624058   Training accuracy   0.8125
Epoch: 0step: 201Training loss:  0.4248095   Training accuracy   0.8125
Epoch: 0step: 202Training loss:  0.38203245   Training accuracy   0.8125
Epoch: 0step: 203Training loss:  0.358968   Training accuracy   0.875
Epoch: 0step: 204Training loss:  0.489994   Training accuracy   0.75
Epoch: 0step: 205Training loss:  0.5196639   Training accuracy   0.75
Epoch: 0step: 206Training loss:  0.55681753   Training accuracy   0.65625
Epoch: 0step: 207Training loss:  0.4542573   Training accuracy   0.78125
Epoch: 0step: 208Training loss:  0.49603164   Training accur

Epoch: 0step: 549Training loss:  0.62312615   Training accuracy   0.6875
Epoch: 0step: 550Training loss:  0.63402665   Training accuracy   0.65625
Epoch: 0step: 551Training loss:  0.6238172   Training accuracy   0.65625
Epoch: 0step: 552Training loss:  0.4392169   Training accuracy   0.84375
Epoch: 0step: 553Training loss:  0.39354163   Training accuracy   0.84375
Epoch: 0step: 554Training loss:  0.47829467   Training accuracy   0.78125
Epoch: 0step: 555Training loss:  0.6592865   Training accuracy   0.59375
Epoch: 0step: 556Training loss:  0.5688541   Training accuracy   0.6875
Epoch: 0step: 557Training loss:  0.5553626   Training accuracy   0.71875
Epoch: 0step: 558Training loss:  0.67195636   Training accuracy   0.5625
Epoch: 0step: 559Training loss:  0.53816605   Training accuracy   0.71875
Epoch: 0step: 560Training loss:  0.515718   Training accuracy   0.78125
Epoch: 0step: 561Training loss:  0.49509278   Training accuracy   0.8125
Epoch: 0step: 562Training loss:  0.57364506   Tra

Epoch: 0step: 663Training loss:  0.59359753   Training accuracy   0.78125
Epoch: 0step: 664Training loss:  0.57621086   Training accuracy   0.6875
Epoch: 0step: 665Training loss:  0.61088896   Training accuracy   0.65625
Epoch: 0step: 666Training loss:  0.6822106   Training accuracy   0.5625
Epoch: 0step: 667Training loss:  0.56283295   Training accuracy   0.6875
Epoch: 0step: 668Training loss:  0.5802913   Training accuracy   0.6875
Epoch: 0step: 669Training loss:  0.60394883   Training accuracy   0.6875
Epoch: 0step: 670Training loss:  0.6060177   Training accuracy   0.71875
Epoch: 0step: 671Training loss:  0.54408014   Training accuracy   0.6875
Epoch: 0step: 672Training loss:  0.6456622   Training accuracy   0.65625
Epoch: 0step: 673Training loss:  0.63302696   Training accuracy   0.59375
Epoch: 0step: 674Training loss:  0.5139559   Training accuracy   0.71875
Epoch: 0step: 675Training loss:  0.5490916   Training accuracy   0.6875
Epoch: 0step: 676Training loss:  0.79726446   Train

Epoch: 0step: 777Training loss:  0.58084506   Training accuracy   0.6875
Epoch: 0step: 778Training loss:  0.5160419   Training accuracy   0.78125
Epoch: 0step: 779Training loss:  0.5783522   Training accuracy   0.75
Epoch: 0step: 780Training loss:  0.5291126   Training accuracy   0.6875
Epoch: 0step: 781Training loss:  0.4788506   Training accuracy   0.8125
Epoch: 0step: 782Training loss:  0.687438   Training accuracy   0.5625
Epoch: 0step: 783Training loss:  0.6195146   Training accuracy   0.75
Epoch: 0step: 784Training loss:  0.46971047   Training accuracy   0.75
Epoch: 0step: 785Training loss:  0.57159287   Training accuracy   0.78125
Epoch: 0step: 786Training loss:  0.62129074   Training accuracy   0.75
Epoch: 0step: 787Training loss:  0.59691185   Training accuracy   0.71875
Epoch: 0step: 788Training loss:  0.7587879   Training accuracy   0.5
Epoch: 0step: 789Training loss:  0.6095028   Training accuracy   0.65625
Epoch: 0step: 790Training loss:  0.6238756   Training accuracy   0.

Epoch: 0step: 891Training loss:  0.6399713   Training accuracy   0.71875
Epoch: 0step: 892Training loss:  0.62755644   Training accuracy   0.65625
Epoch: 0step: 893Training loss:  0.6198348   Training accuracy   0.625
Epoch: 0step: 894Training loss:  0.5780392   Training accuracy   0.71875
Epoch: 0step: 895Training loss:  0.7006804   Training accuracy   0.59375
Epoch: 0step: 896Training loss:  0.50160456   Training accuracy   0.78125
Epoch: 0step: 897Training loss:  0.5821737   Training accuracy   0.75
Epoch: 0step: 898Training loss:  0.47875452   Training accuracy   0.75
Epoch: 0step: 899Training loss:  0.6888284   Training accuracy   0.625
Epoch: 0step: 900Training loss:  0.5905603   Training accuracy   0.625
Epoch: 0step: 901Training loss:  0.6304991   Training accuracy   0.65625
Epoch: 0step: 902Training loss:  0.6955275   Training accuracy   0.5
Epoch: 0step: 903Training loss:  0.58007467   Training accuracy   0.71875
Epoch: 0step: 904Training loss:  0.5909048   Training accuracy 

## Tensorboard event log for Train and validation

In [3]:
count = 0
for e in tf.train.summary_iterator('/home/rohita/rohit/spoof/npy_data_asvspoof/WCNN/Train_whole_dataset_RMS_lr_1e-3/events.out.tfevents.1585538000.anands-MS-7A93'):
    for v in e.summary.value:
        if v.tag == 'loss':
            count = count + 1
            print("Step: "+str(count),v)
        elif v.tag == 'accuracy':
            print(v)


Step: 1 tag: "loss"
simple_value: 0.7553303241729736

tag: "accuracy"
simple_value: 0.59375

Step: 2 tag: "loss"
simple_value: 0.6312949657440186

tag: "accuracy"
simple_value: 0.71875

Step: 3 tag: "loss"
simple_value: 0.9359712600708008

tag: "accuracy"
simple_value: 0.46875

Step: 4 tag: "loss"
simple_value: 0.8375440239906311

tag: "accuracy"
simple_value: 0.5625

Step: 5 tag: "loss"
simple_value: 0.8245865106582642

tag: "accuracy"
simple_value: 0.5

Step: 6 tag: "loss"
simple_value: 0.7479433417320251

tag: "accuracy"
simple_value: 0.5625

Step: 7 tag: "loss"
simple_value: 0.7990843057632446

tag: "accuracy"
simple_value: 0.53125

Step: 8 tag: "loss"
simple_value: 0.8764552474021912

tag: "accuracy"
simple_value: 0.375

Step: 9 tag: "loss"
simple_value: 0.7713212370872498

tag: "accuracy"
simple_value: 0.53125

Step: 10 tag: "loss"
simple_value: 0.7145441770553589

tag: "accuracy"
simple_value: 0.5625

Step: 11 tag: "loss"
simple_value: 0.8268259167671204

tag: "accuracy"
simple_

Step: 238 tag: "loss"
simple_value: 0.692620038986206

tag: "accuracy"
simple_value: 0.5

Step: 239 tag: "loss"
simple_value: 0.6855343580245972

tag: "accuracy"
simple_value: 0.5625

Step: 240 tag: "loss"
simple_value: 0.6867866516113281

tag: "accuracy"
simple_value: 0.5

Step: 241 tag: "loss"
simple_value: 0.679623007774353

tag: "accuracy"
simple_value: 0.59375

Step: 242 tag: "loss"
simple_value: 0.6848911046981812

tag: "accuracy"
simple_value: 0.5

Step: 243 tag: "loss"
simple_value: 0.7025883793830872

tag: "accuracy"
simple_value: 0.46875

Step: 244 tag: "loss"
simple_value: 0.6795462965965271

tag: "accuracy"
simple_value: 0.53125

Step: 245 tag: "loss"
simple_value: 0.705269455909729

tag: "accuracy"
simple_value: 0.46875

Step: 246 tag: "loss"
simple_value: 0.6971195936203003

tag: "accuracy"
simple_value: 0.46875

Step: 247 tag: "loss"
simple_value: 0.6988629698753357

tag: "accuracy"
simple_value: 0.46875

Step: 248 tag: "loss"
simple_value: 0.6880870461463928

tag: "accu

Step: 534 tag: "loss"
simple_value: 0.692594587802887

tag: "accuracy"
simple_value: 0.5

Step: 535 tag: "loss"
simple_value: 0.6896348595619202

tag: "accuracy"
simple_value: 0.5

Step: 536 tag: "loss"
simple_value: 0.6747856736183167

tag: "accuracy"
simple_value: 0.65625

Step: 537 tag: "loss"
simple_value: 0.6828140020370483

tag: "accuracy"
simple_value: 0.5

Step: 538 tag: "loss"
simple_value: 0.6892821788787842

tag: "accuracy"
simple_value: 0.46875

Step: 539 tag: "loss"
simple_value: 0.6749218702316284

tag: "accuracy"
simple_value: 0.53125

Step: 540 tag: "loss"
simple_value: 0.6820090413093567

tag: "accuracy"
simple_value: 0.5

Step: 541 tag: "loss"
simple_value: 0.6767391562461853

tag: "accuracy"
simple_value: 0.5625

Step: 542 tag: "loss"
simple_value: 0.6783318519592285

tag: "accuracy"
simple_value: 0.65625

Step: 543 tag: "loss"
simple_value: 0.6817159652709961

tag: "accuracy"
simple_value: 0.65625

Step: 544 tag: "loss"
simple_value: 0.6738467216491699

tag: "accura

Step: 776 tag: "loss"
simple_value: 0.6815168857574463

tag: "accuracy"
simple_value: 0.625

Step: 777 tag: "loss"
simple_value: 0.698284924030304

tag: "accuracy"
simple_value: 0.4375

Step: 778 tag: "loss"
simple_value: 0.6902457475662231

tag: "accuracy"
simple_value: 0.53125

Step: 779 tag: "loss"
simple_value: 0.6940242052078247

tag: "accuracy"
simple_value: 0.5

Step: 780 tag: "loss"
simple_value: 0.6954923272132874

tag: "accuracy"
simple_value: 0.46875

Step: 781 tag: "loss"
simple_value: 0.6804215312004089

tag: "accuracy"
simple_value: 0.59375

Step: 782 tag: "loss"
simple_value: 0.6886105537414551

tag: "accuracy"
simple_value: 0.5625

Step: 783 tag: "loss"
simple_value: 0.694543719291687

tag: "accuracy"
simple_value: 0.5

Step: 784 tag: "loss"
simple_value: 0.6966633200645447

tag: "accuracy"
simple_value: 0.40625

Step: 785 tag: "loss"
simple_value: 0.6893358826637268

tag: "accuracy"
simple_value: 0.40625

Step: 786 tag: "loss"
simple_value: 0.6626441478729248

tag: "ac

Step: 1091 tag: "loss"
simple_value: 0.7016453146934509

tag: "accuracy"
simple_value: 0.40625

Step: 1092 tag: "loss"
simple_value: 0.6866620182991028

tag: "accuracy"
simple_value: 0.65625

Step: 1093 tag: "loss"
simple_value: 0.6700976490974426

tag: "accuracy"
simple_value: 0.71875

Step: 1094 tag: "loss"
simple_value: 0.6898614764213562

tag: "accuracy"
simple_value: 0.40625

Step: 1095 tag: "loss"
simple_value: 0.6821186542510986

tag: "accuracy"
simple_value: 0.5

Step: 1096 tag: "loss"
simple_value: 0.6906067132949829

tag: "accuracy"
simple_value: 0.5

Step: 1097 tag: "loss"
simple_value: 0.6985718011856079

tag: "accuracy"
simple_value: 0.4375

Step: 1098 tag: "loss"
simple_value: 0.6821402311325073

tag: "accuracy"
simple_value: 0.5625

Step: 1099 tag: "loss"
simple_value: 0.6720393896102905

tag: "accuracy"
simple_value: 0.625

Step: 1100 tag: "loss"
simple_value: 0.6924149394035339

tag: "accuracy"
simple_value: 0.46875

Step: 1101 tag: "loss"
simple_value: 0.6896072626113

Step: 1440 tag: "loss"
simple_value: 0.6943382024765015

tag: "accuracy"
simple_value: 0.5

Step: 1441 tag: "loss"
simple_value: 0.681465744972229

tag: "accuracy"
simple_value: 0.5

Step: 1442 tag: "loss"
simple_value: 0.6953669190406799

tag: "accuracy"
simple_value: 0.4375

Step: 1443 tag: "loss"
simple_value: 0.6873840093612671

tag: "accuracy"
simple_value: 0.5625

Step: 1444 tag: "loss"
simple_value: 0.6907818913459778

tag: "accuracy"
simple_value: 0.4375

Step: 1445 tag: "loss"
simple_value: 0.6874073147773743

tag: "accuracy"
simple_value: 0.59375

Step: 1446 tag: "loss"
simple_value: 0.6752402186393738

tag: "accuracy"
simple_value: 0.625

Step: 1447 tag: "loss"
simple_value: 0.6650974154472351

tag: "accuracy"
simple_value: 0.53125

Step: 1448 tag: "loss"
simple_value: 0.6950926780700684

tag: "accuracy"
simple_value: 0.40625

Step: 1449 tag: "loss"
simple_value: 0.7027908563613892

tag: "accuracy"
simple_value: 0.40625

Step: 1450 tag: "loss"
simple_value: 0.670353949069976

Step: 1800 tag: "loss"
simple_value: 0.7073144912719727

tag: "accuracy"
simple_value: 0.5

Step: 1801 tag: "loss"
simple_value: 0.6932713389396667

tag: "accuracy"
simple_value: 0.53125

Step: 1802 tag: "loss"
simple_value: 0.7025831937789917

tag: "accuracy"
simple_value: 0.40625

Step: 1803 tag: "loss"
simple_value: 0.709053635597229

tag: "accuracy"
simple_value: 0.34375

Step: 1804 tag: "loss"
simple_value: 0.6887661814689636

tag: "accuracy"
simple_value: 0.5

Step: 1805 tag: "loss"
simple_value: 0.6955699920654297

tag: "accuracy"
simple_value: 0.40625

Step: 1806 tag: "loss"
simple_value: 0.6929540634155273

tag: "accuracy"
simple_value: 0.4375

Step: 1807 tag: "loss"
simple_value: 0.6757131814956665

tag: "accuracy"
simple_value: 0.46875

Step: 1808 tag: "loss"
simple_value: 0.6961512565612793

tag: "accuracy"
simple_value: 0.46875

Step: 1809 tag: "loss"
simple_value: 0.6931261420249939

tag: "accuracy"
simple_value: 0.46875

Step: 1810 tag: "loss"
simple_value: 0.70061510801

Step: 2181 tag: "loss"
simple_value: 0.7285856604576111

tag: "accuracy"
simple_value: 0.4375

Step: 2182 tag: "loss"
simple_value: 0.6911428570747375

tag: "accuracy"
simple_value: 0.59375

Step: 2183 tag: "loss"
simple_value: 0.675248384475708

tag: "accuracy"
simple_value: 0.59375

Step: 2184 tag: "loss"
simple_value: 0.6761642694473267

tag: "accuracy"
simple_value: 0.65625

Step: 2185 tag: "loss"
simple_value: 0.6412968039512634

tag: "accuracy"
simple_value: 0.625

Step: 2186 tag: "loss"
simple_value: 0.7081059813499451

tag: "accuracy"
simple_value: 0.375

Step: 2187 tag: "loss"
simple_value: 0.6843562126159668

tag: "accuracy"
simple_value: 0.53125

Step: 2188 tag: "loss"
simple_value: 0.693972647190094

tag: "accuracy"
simple_value: 0.5

Step: 2189 tag: "loss"
simple_value: 0.7060771584510803

tag: "accuracy"
simple_value: 0.375

Step: 2190 tag: "loss"
simple_value: 0.6584111452102661

tag: "accuracy"
simple_value: 0.71875

Step: 2191 tag: "loss"
simple_value: 0.69770276546478

DataLossError: truncated record at 208393926

In [26]:
count = 0
for e in tf.train.summary_iterator('/home/rohita/rohit/spoof/npy_data_asvspoof/MWCNN/Val/events.out.tfevents.1585208859.anands-MS-7A93'):
    for v in e.summary.value:
        if v.tag == 'loss':
            count = count + 1
            print("Step: "+str(count),v)
        elif v.tag == 'accuracy':
            print(v)

Step: 1 tag: "loss"
simple_value: 0.6901168823242188

tag: "accuracy"
simple_value: 0.59375

Step: 2 tag: "loss"
simple_value: 0.6932904124259949

tag: "accuracy"
simple_value: 0.5

Step: 3 tag: "loss"
simple_value: 0.692232608795166

tag: "accuracy"
simple_value: 0.53125

Step: 4 tag: "loss"
simple_value: 0.6954060792922974

tag: "accuracy"
simple_value: 0.4375

Step: 5 tag: "loss"
simple_value: 0.688001275062561

tag: "accuracy"
simple_value: 0.65625

Step: 6 tag: "loss"
simple_value: 0.692232608795166

tag: "accuracy"
simple_value: 0.53125

Step: 7 tag: "loss"
simple_value: 0.692232608795166

tag: "accuracy"
simple_value: 0.53125

Step: 8 tag: "loss"
simple_value: 0.6890591382980347

tag: "accuracy"
simple_value: 0.625

Step: 9 tag: "loss"
simple_value: 0.6932904124259949

tag: "accuracy"
simple_value: 0.5

Step: 10 tag: "loss"
simple_value: 0.6954060792922974

tag: "accuracy"
simple_value: 0.4375

Step: 11 tag: "loss"
simple_value: 0.6985795497894287

tag: "accuracy"
simple_value: 

Step: 175 tag: "loss"
simple_value: 0.6940494775772095

tag: "accuracy"
simple_value: 0.46875

Step: 176 tag: "loss"
simple_value: 0.6932325959205627

tag: "accuracy"
simple_value: 0.5

Step: 177 tag: "loss"
simple_value: 0.6940495371818542

tag: "accuracy"
simple_value: 0.46875

Step: 178 tag: "loss"
simple_value: 0.694866418838501

tag: "accuracy"
simple_value: 0.4375

Step: 179 tag: "loss"
simple_value: 0.690781831741333

tag: "accuracy"
simple_value: 0.59375

Step: 180 tag: "loss"
simple_value: 0.6899648308753967

tag: "accuracy"
simple_value: 0.625

Step: 181 tag: "loss"
simple_value: 0.6940495371818542

tag: "accuracy"
simple_value: 0.46875

Step: 182 tag: "loss"
simple_value: 0.6940494775772095

tag: "accuracy"
simple_value: 0.46875

Step: 183 tag: "loss"
simple_value: 0.68914794921875

tag: "accuracy"
simple_value: 0.65625

Step: 184 tag: "loss"
simple_value: 0.698134183883667

tag: "accuracy"
simple_value: 0.3125

Step: 185 tag: "loss"
simple_value: 0.6973172426223755

tag: "a

Step: 362 tag: "loss"
simple_value: 0.6941164135932922

tag: "accuracy"
simple_value: 0.46875

Step: 363 tag: "loss"
simple_value: 0.6958601474761963

tag: "accuracy"
simple_value: 0.40625

Step: 364 tag: "loss"
simple_value: 0.6949882507324219

tag: "accuracy"
simple_value: 0.4375

Step: 365 tag: "loss"
simple_value: 0.6932445168495178

tag: "accuracy"
simple_value: 0.5

Step: 366 tag: "loss"
simple_value: 0.6949882507324219

tag: "accuracy"
simple_value: 0.4375

Step: 367 tag: "loss"
simple_value: 0.6906288862228394

tag: "accuracy"
simple_value: 0.59375

Step: 368 tag: "loss"
simple_value: 0.6958601474761963

tag: "accuracy"
simple_value: 0.40625

Step: 369 tag: "loss"
simple_value: 0.6932445168495178

tag: "accuracy"
simple_value: 0.5

Step: 370 tag: "loss"
simple_value: 0.6923726797103882

tag: "accuracy"
simple_value: 0.53125

Step: 371 tag: "loss"
simple_value: 0.6949882507324219

tag: "accuracy"
simple_value: 0.4375

Step: 372 tag: "loss"
simple_value: 0.6941163539886475

tag: 

Step: 547 tag: "loss"
simple_value: 0.692614734172821

tag: "accuracy"
simple_value: 0.53125

Step: 548 tag: "loss"
simple_value: 0.692614734172821

tag: "accuracy"
simple_value: 0.53125

Step: 549 tag: "loss"
simple_value: 0.692039966583252

tag: "accuracy"
simple_value: 0.5625

Step: 550 tag: "loss"
simple_value: 0.692614734172821

tag: "accuracy"
simple_value: 0.53125

Step: 551 tag: "loss"
simple_value: 0.6914651989936829

tag: "accuracy"
simple_value: 0.59375

Step: 552 tag: "loss"
simple_value: 0.6908904314041138

tag: "accuracy"
simple_value: 0.625

Step: 553 tag: "loss"
simple_value: 0.6943390369415283

tag: "accuracy"
simple_value: 0.4375

Step: 554 tag: "loss"
simple_value: 0.6954885721206665

tag: "accuracy"
simple_value: 0.375

Step: 555 tag: "loss"
simple_value: 0.6937642693519592

tag: "accuracy"
simple_value: 0.46875

Step: 556 tag: "loss"
simple_value: 0.6954885125160217

tag: "accuracy"
simple_value: 0.375

Step: 557 tag: "loss"
simple_value: 0.6926147937774658

tag: "

Step: 736 tag: "loss"
simple_value: 0.696262776851654

tag: "accuracy"
simple_value: 0.375

Step: 737 tag: "loss"
simple_value: 0.6955024003982544

tag: "accuracy"
simple_value: 0.40625

Step: 738 tag: "loss"
simple_value: 0.6932212114334106

tag: "accuracy"
simple_value: 0.5

Step: 739 tag: "loss"
simple_value: 0.6909400224685669

tag: "accuracy"
simple_value: 0.59375

Step: 740 tag: "loss"
simple_value: 0.6909400224685669

tag: "accuracy"
simple_value: 0.59375

Step: 741 tag: "loss"
simple_value: 0.69474196434021

tag: "accuracy"
simple_value: 0.4375

Step: 742 tag: "loss"
simple_value: 0.6901796460151672

tag: "accuracy"
simple_value: 0.625

Step: 743 tag: "loss"
simple_value: 0.6947420239448547

tag: "accuracy"
simple_value: 0.4375

Step: 744 tag: "loss"
simple_value: 0.6917004585266113

tag: "accuracy"
simple_value: 0.5625

Step: 745 tag: "loss"
simple_value: 0.69474196434021

tag: "accuracy"
simple_value: 0.4375

Step: 746 tag: "loss"
simple_value: 0.69474196434021

tag: "accurac

Step: 928 tag: "loss"
simple_value: 0.6924585103988647

tag: "accuracy"
simple_value: 0.53125

Step: 929 tag: "loss"
simple_value: 0.6916953325271606

tag: "accuracy"
simple_value: 0.5625

Step: 930 tag: "loss"
simple_value: 0.6955114603042603

tag: "accuracy"
simple_value: 0.40625

Step: 931 tag: "loss"
simple_value: 0.6932217478752136

tag: "accuracy"
simple_value: 0.5

Step: 932 tag: "loss"
simple_value: 0.6955114603042603

tag: "accuracy"
simple_value: 0.40625

Step: 933 tag: "loss"
simple_value: 0.690932035446167

tag: "accuracy"
simple_value: 0.59375

Step: 934 tag: "loss"
simple_value: 0.6909320950508118

tag: "accuracy"
simple_value: 0.59375

Step: 935 tag: "loss"
simple_value: 0.6932217478752136

tag: "accuracy"
simple_value: 0.5

Step: 936 tag: "loss"
simple_value: 0.6916953325271606

tag: "accuracy"
simple_value: 0.5625

Step: 937 tag: "loss"
simple_value: 0.6947481632232666

tag: "accuracy"
simple_value: 0.4375

Step: 938 tag: "loss"
simple_value: 0.6909320950508118

tag: "

Step: 1125 tag: "loss"
simple_value: 0.6914960145950317

tag: "accuracy"
simple_value: 0.5625

Step: 1126 tag: "loss"
simple_value: 0.6932450532913208

tag: "accuracy"
simple_value: 0.5

Step: 1127 tag: "loss"
simple_value: 0.6941195726394653

tag: "accuracy"
simple_value: 0.46875

Step: 1128 tag: "loss"
simple_value: 0.6906214952468872

tag: "accuracy"
simple_value: 0.59375

Step: 1129 tag: "loss"
simple_value: 0.6932450532913208

tag: "accuracy"
simple_value: 0.5

Step: 1130 tag: "loss"
simple_value: 0.6949940919876099

tag: "accuracy"
simple_value: 0.4375

Step: 1131 tag: "loss"
simple_value: 0.6897469162940979

tag: "accuracy"
simple_value: 0.625

Step: 1132 tag: "loss"
simple_value: 0.6941195726394653

tag: "accuracy"
simple_value: 0.46875

Step: 1133 tag: "loss"
simple_value: 0.6949940919876099

tag: "accuracy"
simple_value: 0.4375

Step: 1134 tag: "loss"
simple_value: 0.6932450532913208

tag: "accuracy"
simple_value: 0.5

Step: 1135 tag: "loss"
simple_value: 0.6914960145950317



Step: 1328 tag: "loss"
simple_value: 0.6964192390441895

tag: "accuracy"
simple_value: 0.46875

Step: 1329 tag: "loss"
simple_value: 0.6964192390441895

tag: "accuracy"
simple_value: 0.46875

Step: 1330 tag: "loss"
simple_value: 0.6964192390441895

tag: "accuracy"
simple_value: 0.46875

Step: 1331 tag: "loss"
simple_value: 0.6765543222427368

tag: "accuracy"
simple_value: 0.71875

Step: 1332 tag: "loss"
simple_value: 0.6964192390441895

tag: "accuracy"
simple_value: 0.46875

Step: 1333 tag: "loss"
simple_value: 0.7038686275482178

tag: "accuracy"
simple_value: 0.375

Step: 1334 tag: "loss"
simple_value: 0.6964192390441895

tag: "accuracy"
simple_value: 0.46875

Step: 1335 tag: "loss"
simple_value: 0.6939361691474915

tag: "accuracy"
simple_value: 0.5

Step: 1336 tag: "loss"
simple_value: 0.701385498046875

tag: "accuracy"
simple_value: 0.40625

Step: 1337 tag: "loss"
simple_value: 0.6914530396461487

tag: "accuracy"
simple_value: 0.53125

Step: 1338 tag: "loss"
simple_value: 0.70138549

Step: 1509 tag: "loss"
simple_value: 0.6931157112121582

tag: "accuracy"
simple_value: 0.53125

Step: 1510 tag: "loss"
simple_value: 0.6930210590362549

tag: "accuracy"
simple_value: 0.625

Step: 1511 tag: "loss"
simple_value: 0.6931473016738892

tag: "accuracy"
simple_value: 0.5

Step: 1512 tag: "loss"
simple_value: 0.6932419538497925

tag: "accuracy"
simple_value: 0.40625

Step: 1513 tag: "loss"
simple_value: 0.6933682560920715

tag: "accuracy"
simple_value: 0.28125

Step: 1514 tag: "loss"
simple_value: 0.6933998465538025

tag: "accuracy"
simple_value: 0.25

Step: 1515 tag: "loss"
simple_value: 0.6930525898933411

tag: "accuracy"
simple_value: 0.59375

Step: 1516 tag: "loss"
simple_value: 0.6931157112121582

tag: "accuracy"
simple_value: 0.53125

Step: 1517 tag: "loss"
simple_value: 0.6930526494979858

tag: "accuracy"
simple_value: 0.59375

Step: 1518 tag: "loss"
simple_value: 0.6932420134544373

tag: "accuracy"
simple_value: 0.40625

Step: 1519 tag: "loss"
simple_value: 0.6930209994